# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1f60199c10>
├── 'a' --> tensor([[-0.2704,  1.6590, -0.7301],
│                   [-0.3963,  1.5045,  0.7599]])
└── 'x' --> <FastTreeValue 0x7f1f60199460>
    └── 'c' --> tensor([[ 0.7411, -1.4261,  0.7883,  1.4179],
                        [ 0.6141,  0.4478, -0.0814, -0.8131],
                        [-0.2000,  0.7481,  0.0197,  1.4457]])

In [4]:
t.a

tensor([[-0.2704,  1.6590, -0.7301],
        [-0.3963,  1.5045,  0.7599]])

In [5]:
%timeit t.a

71.5 ns ± 1.62 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1f60199c10>
├── 'a' --> tensor([[ 0.0829, -0.4415,  0.9690],
│                   [ 0.5901, -0.5109, -1.2168]])
└── 'x' --> <FastTreeValue 0x7f1f60199460>
    └── 'c' --> tensor([[ 0.7411, -1.4261,  0.7883,  1.4179],
                        [ 0.6141,  0.4478, -0.0814, -0.8131],
                        [-0.2000,  0.7481,  0.0197,  1.4457]])

In [7]:
%timeit t.a = new_value

75.8 ns ± 1.04 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2704,  1.6590, -0.7301],
               [-0.3963,  1.5045,  0.7599]]),
    x: Batch(
           c: tensor([[ 0.7411, -1.4261,  0.7883,  1.4179],
                      [ 0.6141,  0.4478, -0.0814, -0.8131],
                      [-0.2000,  0.7481,  0.0197,  1.4457]]),
       ),
)

In [10]:
b.a

tensor([[-0.2704,  1.6590, -0.7301],
        [-0.3963,  1.5045,  0.7599]])

In [11]:
%timeit b.a

65.9 ns ± 0.846 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0609,  0.3229, -0.2893],
               [-1.6343, -0.4445, -0.4575]]),
    x: Batch(
           c: tensor([[ 0.7411, -1.4261,  0.7883,  1.4179],
                      [ 0.6141,  0.4478, -0.0814, -0.8131],
                      [-0.2000,  0.7481,  0.0197,  1.4457]]),
       ),
)

In [13]:
%timeit b.a = new_value

610 ns ± 14.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.01 µs ± 32.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.6 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

288 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

271 µs ± 7.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1ea844cd30>
├── 'a' --> tensor([[[-0.2704,  1.6590, -0.7301],
│                    [-0.3963,  1.5045,  0.7599]],
│           
│                   [[-0.2704,  1.6590, -0.7301],
│                    [-0.3963,  1.5045,  0.7599]],
│           
│                   [[-0.2704,  1.6590, -0.7301],
│                    [-0.3963,  1.5045,  0.7599]],
│           
│                   [[-0.2704,  1.6590, -0.7301],
│                    [-0.3963,  1.5045,  0.7599]],
│           
│                   [[-0.2704,  1.6590, -0.7301],
│                    [-0.3963,  1.5045,  0.7599]],
│           
│                   [[-0.2704,  1.6590, -0.7301],
│                    [-0.3963,  1.5045,  0.7599]],
│           
│                   [[-0.2704,  1.6590, -0.7301],
│                    [-0.3963,  1.5045,  0.7599]],
│           
│                   [[-0.2704,  1.6590, -0.7301],
│                    [-0.3963,  1.5045,  0.7599]]])
└── 'x' --> <FastTreeValue 0x7f1ea844cd00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.6 µs ± 562 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1f6020ffd0>
├── 'a' --> tensor([[-0.2704,  1.6590, -0.7301],
│                   [-0.3963,  1.5045,  0.7599],
│                   [-0.2704,  1.6590, -0.7301],
│                   [-0.3963,  1.5045,  0.7599],
│                   [-0.2704,  1.6590, -0.7301],
│                   [-0.3963,  1.5045,  0.7599],
│                   [-0.2704,  1.6590, -0.7301],
│                   [-0.3963,  1.5045,  0.7599],
│                   [-0.2704,  1.6590, -0.7301],
│                   [-0.3963,  1.5045,  0.7599],
│                   [-0.2704,  1.6590, -0.7301],
│                   [-0.3963,  1.5045,  0.7599],
│                   [-0.2704,  1.6590, -0.7301],
│                   [-0.3963,  1.5045,  0.7599],
│                   [-0.2704,  1.6590, -0.7301],
│                   [-0.3963,  1.5045,  0.7599]])
└── 'x' --> <FastTreeValue 0x7f1f6020f160>
    └── 'c' --> tensor([[ 0.7411, -1.4261,  0.7883,  1.4179],
                        [ 0.6141,  0.4478, -0.0814, -0.8131],
                 

In [23]:
%timeit t_cat(trees)

43.6 µs ± 749 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

80.3 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2704,  1.6590, -0.7301],
                [-0.3963,  1.5045,  0.7599]],
       
               [[-0.2704,  1.6590, -0.7301],
                [-0.3963,  1.5045,  0.7599]],
       
               [[-0.2704,  1.6590, -0.7301],
                [-0.3963,  1.5045,  0.7599]],
       
               [[-0.2704,  1.6590, -0.7301],
                [-0.3963,  1.5045,  0.7599]],
       
               [[-0.2704,  1.6590, -0.7301],
                [-0.3963,  1.5045,  0.7599]],
       
               [[-0.2704,  1.6590, -0.7301],
                [-0.3963,  1.5045,  0.7599]],
       
               [[-0.2704,  1.6590, -0.7301],
                [-0.3963,  1.5045,  0.7599]],
       
               [[-0.2704,  1.6590, -0.7301],
                [-0.3963,  1.5045,  0.7599]]]),
    x: Batch(
           c: tensor([[[ 0.7411, -1.4261,  0.7883,  1.4179],
                       [ 0.6141,  0.4478, -0.0814, -0.8131],
                       [-0.2000,  0.7481,  0.0197,  1.4457]],
         

In [26]:
%timeit Batch.stack(batches)

104 µs ± 2.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2704,  1.6590, -0.7301],
               [-0.3963,  1.5045,  0.7599],
               [-0.2704,  1.6590, -0.7301],
               [-0.3963,  1.5045,  0.7599],
               [-0.2704,  1.6590, -0.7301],
               [-0.3963,  1.5045,  0.7599],
               [-0.2704,  1.6590, -0.7301],
               [-0.3963,  1.5045,  0.7599],
               [-0.2704,  1.6590, -0.7301],
               [-0.3963,  1.5045,  0.7599],
               [-0.2704,  1.6590, -0.7301],
               [-0.3963,  1.5045,  0.7599],
               [-0.2704,  1.6590, -0.7301],
               [-0.3963,  1.5045,  0.7599],
               [-0.2704,  1.6590, -0.7301],
               [-0.3963,  1.5045,  0.7599]]),
    x: Batch(
           c: tensor([[ 0.7411, -1.4261,  0.7883,  1.4179],
                      [ 0.6141,  0.4478, -0.0814, -0.8131],
                      [-0.2000,  0.7481,  0.0197,  1.4457],
                      [ 0.7411, -1.4261,  0.7883,  1.4179],
                      [ 0.6141,  

In [28]:
%timeit Batch.cat(batches)

246 µs ± 5.99 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

591 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
